In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import igpu
import cupy as cp
import pickle

In [2]:
mice = ["mpfc14", "mpfc19", "mpfc20", "mpfc23", "mpfc24", "mpfc30"]
days = ["yd1", "yd2", "yd6"]

In [3]:
x = []
h = []
w = []
for mi, mouse in enumerate(mice):
    ws = []
    hs = []
    xs = []
    for di, day in enumerate(days):
        i = 0
        wss = []
        while True:
            try:
                df = pd.read_csv(mouse + "/" + mouse + "_" + day + "_igpu_Wlim001_W" + str(i) + ".csv", header=None).values
                wss.append(df)
                i += 1
            except:
                break
        ws.append(wss)
        hs.append(pd.read_csv(mouse + "/" + mouse + "_" + day + "_igpu_Wlim001_H.csv", header=None).values)
        xs.append(pd.read_csv(mouse + "/" + mouse + "_" + day + ".csv", header=None).values)
    w.append(ws)
    h.append(hs)
    x.append(xs)

In [5]:
w_single = []
h_single = []
for mi, _ in enumerate(mice):
    ws = []
    hs = []
    for di, _ in enumerate(days):
        h_tmp = np.zeros_like(h[mi][di])
        t_h = h_tmp.shape[0]
        n, l = w[mi][di][0].shape
        win = l - 1
        win2 = 2 * l - 1
        wss = []
        for wi in range(h[mi][di].shape[1]):
            print(mi, di, wi)
            u = cp.asnumpy(igpu.conv(cp.asarray(w[mi][di][wi][:, cp.newaxis, :]), cp.asarray(h[mi][di][:, wi:wi+1]).T, x[mi][di][x[mi][di] > 0].min()))
            umu = u - u.mean(axis=1)[:, np.newaxis]
            umu2 = umu * umu
            i = u.sum(axis=1).argmax()
            tmp = np.zeros((n, win2))
            for j in range(n):
                if j == i:
                    tmp[i, win] = 1
                    continue
                den2i0 = umu2[i].sum()
                den2j0 = umu2[j].sum()
                if den2j0 == 0:
                    continue
                tmp[j, win] = (umu[i] * umu[j]).sum() / np.sqrt(den2i0 * den2j0)
                den2i = den2i0
                den2j = den2j0
                for p in range(1, l):
                    den2i -= umu2[i, p-1]
                    den2j -= umu2[j, -p]
                    tmp[j, win - p] = (umu[i, p:] * umu[j, :-p]).sum() / np.sqrt(den2i * den2j)
                den2i = den2i0
                den2j = den2j0
                for p in range(1, l):
                    den2i -= umu2[i, -p]
                    den2j -= umu2[j, p-1]
                    tmp[j, win + p] = (umu[i, :-p] * umu[j, p:]).sum() / np.sqrt(den2i * den2j)
            index = tmp.argmax(axis=1)
            index_count = np.array([(index == i).sum() for i in range(win2)])
            scand = np.zeros(win)
            c = index_count[:win].sum()
            for s in range(win):
                c += index_count[s+win]
                scand[s] = c
                c -= index_count[s]
            start = scand.argmax()
            w_pos = tmp[:, start:start+l].argmax(axis=1)
            w_act = u.sum(axis=1) / u.sum()
            h_raw = u / w_act[:, np.newaxis]
            h_count = np.zeros(t_h)
            for i, (a, p) in enumerate(zip(w_act, w_pos)):
                h_tmp[win-p:t_h-p, wi] += a * h_raw[i]
                h_count[win-p:t_h-p] += a
            h_count[h_count == 0] = 1
            h_tmp[:, wi] /= h_count
            wss.append(np.vstack((w_act, w_pos)).T)
        hs.append(h_tmp)
        ws.append(wss)
    h_single.append(hs)
    w_single.append(ws)

0 0 0
0 0 1
0 0 2
0 0 3
0 0 4
0 0 5
0 1 0
0 1 1
0 1 2
0 1 3
0 1 4
0 1 5
0 2 0
0 2 1
0 2 2
0 2 3
0 3 0
0 3 1
0 3 2
0 3 3
0 3 4
0 3 5
0 4 0
0 4 1
0 4 2
0 4 3
0 4 4
0 5 0
0 5 1
0 5 2
0 5 3
1 0 0
1 0 1
1 0 2
1 0 3
1 0 4
1 0 5
1 0 6
1 0 7
1 0 8
1 1 0
1 1 1
1 1 2
1 1 3
1 1 4
1 2 0
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
1 2 7
1 2 8
1 2 9
1 3 0
1 3 1
1 3 2
1 3 3
1 3 4
1 4 0
1 4 1
1 4 2
1 4 3
1 4 4
1 4 5
1 4 6
1 4 7
1 5 0
1 5 1
1 5 2
1 5 3
1 5 4
1 5 5
1 5 6
2 0 0
2 0 1
2 0 2
2 0 3
2 0 4
2 0 5
2 0 6
2 1 0
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 2 0
2 2 1
2 2 2
2 2 3
2 2 4
2 2 5
2 2 6
2 3 0
2 3 1
2 3 2
2 3 3
2 3 4
2 3 5
2 3 6
2 4 0
2 4 1
2 4 2
2 4 3
2 4 4
2 4 5
2 4 6
2 4 7
2 5 0
2 5 1
2 5 2
2 5 3
2 5 4
2 5 5
2 5 6
3 0 0
3 0 1
3 0 2
3 0 3
3 0 4
3 0 5
3 0 6
3 1 0
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 2 0
3 2 1
3 2 2
3 2 3
3 2 4
3 2 5
3 3 0
3 3 1
3 3 2
3 3 3
3 3 4
3 4 0
3 4 1
3 4 2
3 4 3
3 4 4
3 4 5
3 4 6
3 5 0
3 5 1
3 5 2
3 5 3
4 0 0
4 0 1
4 0 2
4 0 3
4 0 4
4 1 0
4 1 1
4 1 2
4 1 3
4 2 0
4 2 1
4 2 2
4 2 3
4 2 4
4 2 

In [6]:
with open("w_single_Wlim001.pkl", "wb") as f:
    pickle.dump(w_single, f)
with open("h_single_Wlim001.pkl", "wb") as f:
    pickle.dump(h_single, f)

In [7]:
h_spike = []
for mi, mouse in enumerate(mice):
    spikes = []
    for di, day in enumerate(days):
        move_ave = np.zeros_like(h_single[mi][di])
        s = h_single[mi][di][:49].sum(axis=0)
        num = 49
        t = move_ave.shape[0]
        for i in range(t):
            if i + 49 < t:
                num += 1
                s[:] += h_single[mi][di][i+49, :]
            move_ave[i, :] = s[:] / num
            if i - 49 > 0:
                num -= 1
                s[:] -= h_single[mi][di][i-49, :]
        tmp = h_single[mi][di] - move_ave
        tmp[tmp < 0] = 0
        th =  tmp.mean(axis=0) + 2 * tmp.std(axis=0)
        for i, thi in enumerate(th):
            tmp[:, i][tmp[:, i] > thi] = thi
            tmp[:, i] /= thi
        spikes.append(tmp)
    h_spike.append(spikes)
    
with open("h_spike_Wlim001.pkl", "wb") as f:
    pickle.dump(h_spike, f)